In [ ]:
#make directory and copy the data
!mkdir SQUAD_DIR && cd SQUAD_DIR && wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

mkdir: cannot create directory ‘SQUAD_DIR’: File exists


In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install --upgrade pip

!pip install --upgrade google-api-python-client

     |████████████████████████████████| 2.1 MB 490 kB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 KB 11.4 MB/s eta 0:00:00
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.0.4
    Uninstalling google-auth-httplib2-0.0.4:
      Successfully uninstalled google-auth-httplib2-0.0.4
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.26.3
    Uninstalling google-api-core-1.26.3:
      Successfully uninstalled google-api-core-1.26.3
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11
ERROR: pip's

In [ ]:
%%writefile back_translate.py
# --------------- Produce SQUAD 2.0 augmented dataset --------------------------------------------------
#  Description: Augment Questions in SQUAD 2.0 dataset with syntatic variance produced by Back Translation
#               Using Neural Machine Translation System (NMT)
#               Designed to work with Google Cloud Translate API
#               The API Translates the original question (English to German) and then back translates (German to English)


import json
import random
from math import floor
import secrets
import os

with open('/content/SQUAD_DIR/train-v2.0.json',encoding='utf-8') as f:
    data = json.load(f)
from google.cloud import translate

credential_path = "/content/drive/MyDrive/QA/river-autumn-337604-00ce048c68e7.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path


client = translate.Client()

def randomly_replace_synonym(tmp_in,percentage=0.5):  
    len_in = len(tmp_in)
    #Deep Copy
    tmp_in_ip = tmp_in[:]

    if True:
        #Translate En -> De
        target = 'de'
        translation = client.translate(tmp_in,target_language=target)
        trans_output = translation["translatedText"].replace('&#39;',"'")
        trans_output = trans_output.replace('&quot;',"'")

        #Translate De -> En
        target = 'en'
        translation_en = client.translate(trans_output,target_language=target)
        tmp_out = translation_en["translatedText"].replace('&#39;',"'").replace('&quot;', "'")
        tmp_out = tmp_out.encode('utf8')

    return tmp_out
     


for i in range(0,len(data["data"])):
    for j in range(0,len(data["data"][i]["paragraphs"])):
        for k in range(0,len(data["data"][i]["paragraphs"][j]["qas"])):
            tmp_in = data["data"][i]["paragraphs"][j]["qas"][k]["question"]
            #print('Og is',tmp_in)
            tmp_out_list = randomly_replace_synonym(tmp_in)
            #print('before',len(data["data"][i]["paragraphs"][j]["qas"]))
            #print(tmp_out_list)
            if tmp_out_list != '':
                dict = data["data"][i]["paragraphs"][j]["qas"][k]
                dict["id"] = secrets.token_hex(15)
                dict["question"] = tmp_out_list.decode('utf8').replace("'", '"')
                data["data"][i]["paragraphs"][j]["qas"].append(dict)
                print('after',len(data["data"][i]["paragraphs"][j]["qas"]))
                print('paragraph number is', j)

print('Writing Final JSON')
with open('train-v2-aug.json', 'w') as outfile:
    json.dump(data, outfile)              

Overwriting back_translate.py


In [ ]:
!python back_translate.py

Streaming output truncated to the last 5000 lines.
paragraph number is 41
after 9
paragraph number is 41
after 10
paragraph number is 41
after 6
paragraph number is 42
after 7
paragraph number is 42
after 8
paragraph number is 42
after 9
paragraph number is 42
after 10
paragraph number is 42
after 5
paragraph number is 43
after 6
paragraph number is 43
after 7
paragraph number is 43
after 8
paragraph number is 43
after 6
paragraph number is 44
after 7
paragraph number is 44
after 8
paragraph number is 44
after 9
paragraph number is 44
after 10
paragraph number is 44
after 6
paragraph number is 45
after 7
paragraph number is 45
after 8
paragraph number is 45
after 9
paragraph number is 45
after 10
paragraph number is 45
after 6
paragraph number is 46
after 7
paragraph number is 46
after 8
paragraph number is 46
after 9
paragraph number is 46
after 10
paragraph number is 46
after 6
paragraph number is 47
after 7
paragraph number is 47
after 8
paragraph number is 47
after 9
paragraph numb

In [7]:
import json
import random
from math import floor
import secrets
import os
from google.cloud import translate

credential_path = "/content/drive/MyDrive/QA/river-autumn-337604-00ce048c68e7.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

client = translate.Client()
#Translate En -> De
target = 'de'
translation = client.translate('What artist did Beyonce duet with in the single',target_language=target)
print(translation)

{'translatedText': 'Mit welchem Künstler duettierte sich Beyonce in der Single?', 'detectedSourceLanguage': 'en', 'input': 'What artist did Beyonce duet with in the single'}


In [8]:
#Translate De -> En
target = 'en'
translation_en = client.translate('Mit welchem Künstler duettierte sich Beyonce in der Single?',target_language=target)
print(translation_en)

{'translatedText': 'Which artist did Beyonce duet with on the single?', 'detectedSourceLanguage': 'de', 'input': 'Mit welchem Künstler duettierte sich Beyonce in der Single?'}
